# Парсинг OPUS
Делим файл на два файла: строчка 1 файла соответствует строчке 2 файла

In [1]:
from tqdm import tqdm_notebook as tqdm
from lxml import etree
import re

class CleanCorpus:
     
    def __init__(self):
        self.TMX_FILE = ''
        self.lang_1 = ''
        self.lang_2 = ''
        
    def find_langs(self):
        langs = re.search(r'(.+?)-(.+?).tmx', self.TMX_FILE)
        if langs:
            self.lang_1 = langs.group(1)
            self.lang_2 = langs.group(2)
        else:
            self.lang_1 = 'in'
            self.lang_2 = 'out'
            
    def parse_tmx_opus_file(self, TMX_FILE):
        """
        Write into two files: lang_1.txt and lang_2.txt
        """
        self.TMX_FILE = TMX_FILE
        self.find_langs()
        context = etree.iterparse(self.TMX_FILE, tag='tu')
        context = iter(context)
        with open(self.lang_1 + '.txt', 'a') as file_1, open(self.lang_2 + '.txt', 'a') as file_2:
            for event, elem in tqdm(context):
                en_ru = elem.findall('tuv')
                en = en_ru[0]
                ru = en_ru[1]
                seg_en = en.find('seg')
                seg_ru = ru.find('seg')
                en_text = seg_en.text
                ru_text = seg_ru.text
                file_1.write(en_text + '\n\n')
                file_2.write(ru_text + '\n\n')

In [2]:
TMX_FILE = 'en-ru.tmx'
CC = CleanCorpus()
CC.parse_tmx_opus_file(TMX_FILE)

# Парсинг UDPipe

На вход - целый файл одного языка, на выходе файл строчка: ```token_1 token2 ... token_n```

In [1]:
from parsing import ParserUDpipe

In [2]:
MODEL = 'english-partut-ud-2.3-181115.udpipe'

In [2]:
#MODEL = 'english-partut-ud-2.3-181115.udpipe'
#PATH = 'en.txt'

In [3]:
from os import listdir
from os.path import isfile, join
mypath = 'en'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
onlyfiles = onlyfiles[0:2]
onlyfiles

['en_100000.txt', 'en_1000000.txt']

In [ ]:
for x in onlyfiles:
    parser = ParserUDpipe()
    parser.parsing2conllu('./en/' + x, 'conllu_' + x, MODEL)
    parser.lines2tokens('conllu_' + x, 'token_en', token='form')

Parsing ./en/en_100000.txt


In [4]:
parser.lines2tokens('conllu_my.txt', token='lemma')


Write into my_lemma_string.txt


In [5]:
parser.lines2tokens('conllu_my.txt', token='form')


Write into my_form_string.txt


In [6]:
parser.lines2tokens('conllu_my.txt', token='upostag')


Write into my_upostag_string.txt


In [5]:
lines_per_file = 100000
smallfile = None
with open('en.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        if lineno % lines_per_file == 0:
            if smallfile:
                smallfile.close()
            small_filename = './en/en_{}.txt'.format(lineno + lines_per_file)
            smallfile = open(small_filename, "w")
        smallfile.write(line)
    if smallfile:
        smallfile.close()